## Merging DataFrames
Inthis notebook we're going to address how you can bring multiple dataframe objects together, either by merging them horizontally, or by concatenating them vertically. Before we jump into the code, we need to address a little relational theory and to get some language convetions down. A Venn Diagram is traditionally used to show set membership. For example, the circle on the left is the population of students at a university. The circle on the right is the population of staff at university. And the overlapping regio un this middle are all of those students who are also staff. Maybe these students run tutorials for a course, or grade assignments, or engage in running research experiments. So, this diagram shows two populations whom we nught have data about, but there is overlap between those populations.
When it comes to translation this pandas, we can think of the case where we might have these two populations as indices in separate DataFrames, maybe with the label of PersonName. When we want to join the DataFrames together, we have some choices to maje, First what if we want a listo of all the people regardless of whether they're staff or student, and all of the inforamtion we can get of them? In database terminology, this is called a full outer join. And in ser theory, it's called a union. In the Venn diagram, it represents everyone in any circle.
FULL OUTER JOIN (UNION) All elements of both sets.
It's quite possible though that we only want those people who we hace maximum information for, these people who are both staff and students. Maybe being a staff member and a studen involves getting a tuition waiver, and we want to calculate the cost of this. In database terminology, this is called an inner join. Or in set theory, the intersection.
INNER JOIN (INTERSECTION) Only elments are in both sets.

In [1]:
#Let's to create two dataframes for explore the diferents forms to merging it.
import pandas as pd
import numpy as np
import timeit

staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
staff_df.set_index('Name', inplace = True)

student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])

student_df.set_index('Name', inplace = True)

print(staff_df)
print(student_df)
# There's some overlap in these DataFrames, in that James and Sally are both students and staff, but Mike and Kelly are not. Importantly, both DataFrames are indexed along the value we want to
# merge them on, which is called Name.

                 Role
Name                 
Kelly  Director of HR
Sally  Course liasion
James          Grader
            School
Name              
James     Business
Mike           Law
Sally  Engineering


In [2]:
# If we want the union of these, we would call merge() passing in the DataFrame on the left and the DataFrame on the right and telling merge that we want it to use an outer join.
# We want to use the left and right indices as the joining columns.
pd.merge(staff_df, student_df, how = 'outer', left_index = True, right_index = True) # left_index and reight index are for use both indexes to join dataframes

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


In [3]:
# We see in the resulting DataFrame that everyone is listed. And since Mike dos not have a role, and John does not have a school those cells are listed as missing values.
# If we wanted to ge the intersection, that is, just those who are a student AND a staff, we could ser the how attribute to inner. Again, we set both left and right indices to be true as the joining cols.
pd.merge(staff_df, student_df, how = 'inner', left_index = True, right_index = True)

,Role,School
Name,,
Sally,Course liasion,Engineering
James,Grader,Business


In [4]:
# And we see the resulting DataFrame has only James and Sally in it. Now there are two other common use cases whe merging DataFrames, and both are examples of what we would call set addition. 
# The first is when we would want to get a list of all staff regardless of wether they were students or not. Buy if ther were students, we would want to get their student details as well. To do this
# we would use a left join. It is important to note the order of dataframes in this function: the first dataframe is the left dataframe and the second is the right.
pd.merge(staff_df, student_df, how = 'left', left_index = True, right_index = True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course liasion,Engineering
James,Grader,Business


In [5]:
# We could probably guess what comes next. We want a list of all of the students and their roles if they were also staff. To do this we would do a right join.
pd.merge(staff_df, student_df, how = 'right', left_index = True, right_index = True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


In [6]:
# We can also do it another way. The merge method has a couple of other interesting parameters. First, you don't need to use indeces to join on, you can use columns as well. Here's an example.
# Here we have a parameter called "on", and we can assign a column that both dataframe has the joining column.
# First, lets remove our index from both of our dataframes
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()

# Now lets merge using the on parameter
pd.merge(staff_df, student_df, how='right', on='Name')

,Name,Role,School
0,James,Grader,Business
1,Mike,NaN,Law
2,Sally,Course liasion,Engineering


In [7]:
# Using the "on" parameter instead of a the index is how I find myself usign merge() the most. So what happens when we have conflicts between the DataFrames? Let's try take a look by creating new staff 
# and student DataFrames that have a location information added to them.
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 'Location': '512 Wilson Crescent'}])
# In this staff DataFrame, this is an office location where we can find the staff person. And we can see the Director of HR is on Satate Street, while the two srtudents are on Washington Avenue, and
# these locations just happen to be right outside my window as I film this. But for the studen DataFrame , the location information is actually their home address.
# Ther merge function preserves this informations, but appends and _x or _y to help differentiate betwwen which index went which column of data. The _x is always the left DataFrame information, and
# the _y is always the right DataFrame information.
# Here, if we want all the staff information regardless of whether they were students or not. But if they were students, we could want to get their student details as well. Then we can do a left join
# and on the column of Name
pd.merge(staff_df, student_df, how = 'left', on = 'Name')
# From the outpout, we can see there are columns locations_x and location_y. Location x refers to the location column in the left dataframe which is staff dataframe and location_y refers to the location
#column in the right dataFrame, which is studen dataFrame.

,Name,Role,Location_x,School,Location_y
0,Kelly,Director of HR,State Street,NaN,NaN
1,Sally,Course liasion,Washington Avenue,Engineering,512 Wilson Crescent
2,James,Grader,Washington Avenue,Business,1024 Billiard Avenue


In [8]:
# Before we leave merging DataFrames, Let's talk about multi - indexing and mulple columns. In's quite possible that the first name for students and staff might overla but the last name might not. 
# In his case, we use a list of the multiple columns that should be used to join keys from both dataframes on the on parameter. Recall that the column name (s) assigned to the on parameter needs
# to exist in both dataframes.
# Here's an example with some new student and staff data
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 
                          'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 
                          'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 
                          'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 
                            'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 
                            'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 
                            'School': 'Engineering'}])
# AS you see here, James Wilde and James Hammond don't match on both keys since they have different last names. So we could expect that an inner join doesn't include these individuals in the output
# an only Sally Brooks will be retained
pd.merge(staff_df, student_df, how = 'inner', on = ['First Name', 'Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


## Python idioms
### Pandorable
Python programmers will often suggest that there many ways the language can be used to solve a particular problem. But that some more appropriate than others. The best solutions are celebrated as Idiomatic Python and there are lots of great examples of this on StackOverflow and other websites.
A sort of sub-language within Python, Pandas has its owm set idioms. We've alluded to some of these already, such as using vectorization whenever possible, and not using iterative loops if you don't need to. Several developers and users within the Panda's community have used the term pandorable for these idioms. I think it's a great term. So, I wanted to share with you a couple of key features of how you can make your code pandorable.

In [16]:
df = pd.read_csv('data/census.csv')
# The first of the pandas idioms I would like to talk about is called method chaining. The general idea behind method chaining is that every method on an object returns a reference to that object. 
# The beauty of this is that you can condense many different operations on a DataFrame, for instance, into one line or at least one statement of code. 
# Here's the pandorable way to write code with method chaining. In this code I'm going to pull out the state and city names as a multiple index, and I'm going to do so only for data which has a
# summary level of 50, which in this dataset is county - level data. I'll rename a column too, just to make it a bit more readable.
(df.where(df['SUMLEV'] == 50)
   .dropna()
   .set_index(['STNAME', 'CTYNAME'])
   .rename(columns = {'ESTIMATESBASE2010': 'Estimates Base 2010'}))
# Lets walk through this.First, we use the where() function on the dataframe and pass in a boolean mask which is only true for those rows where SUMLE is equal to 50. This indicates in our source data
# is summarized at the conuty level. With te result of where() function evaluated, we drop missing values. Remember that .where() doesn't drop missing values by default. The we set an index on the
# result of that. In this case I've set it to the state name followed by the county name. Finally, I rename a column to mmake it more readable. Note that instead od writing this all on one line,
# as I could have done, I bega the statemen with parenthesus, which tell python I'm going to span the statemen over multiple lines for readability.
df[df['SUMLEV']==50].set_index(['STNAME','CTYNAME']).rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}) # this is alternative way, only in one line and use [] operator with boolean mask
# instaed of where and dropna

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                      
Alabama Autauga County         50       3         6      1       1   
        Baldwin County         50       3         6      1       3   
        Barbour County         50       3         6      1       5   
        Bibb County            50       3         6      1       7   
        Blount County          50       3         6      1       9   
...                           ...     ...       ...    ...     ...   
Wyoming Sweetwater County      50       4         8     56      37   
        Teton County           50       4         8     56      39   
        Uinta County           50       4         8     56      41   
        Washakie County        50       4         8     56      43   
        Weston County          50       4         8     56      45   

                           CENSUS2010POP  Estimates Base 2010  \
STNAME  CTYNAME                                                 
Alabama Autauga County             54571                54571   
        Baldwin County            182265               182265   
        Barbour County             27457                27457   
        Bibb County                22915                22919   
        Blount County              57322                57322   
...                                  ...                  ...   
Wyoming Sweetwater County          43806                43806   
        Teton County               21294                21294   
        Uinta County               21118                21118   
        Washakie County             8533                 8533   
        Weston County               7208                 7208   

                           POPESTIMATE2010  POPESTIMATE2011  POPESTIMATE2012  \
STNAME  CTYNAME                                                                
Alabama Autauga County               54660            55253            55175   
        Baldwin County              183193           186659           190396   
        Barbour County               27341            27226            27159   
        Bibb County                  22861            22733            22642   
        Blount County                57373            57711            57776   
...                                    ...              ...              ...   
Wyoming Sweetwater County            43593            44041            45104   
        Teton County                 21297            21482            21697   
        Uinta County                 21102            20912            20989   
        Washakie County               8545             8469             8443   
        Weston County                 7181             7114             7065   

                           ...  RDOMESTICMIG2011  RDOMESTICMIG2012  \
STNAME  CTYNAME            ...                                       
Alabama Autauga County     ...          7.242091         -2.915927   
        Baldwin County     ...         14.832960         17.647293   
        Barbour County     ...         -4.728132         -2.500690   
        Bibb County        ...         -5.527043         -5.068871   
        Blount County      ...          1.807375         -1.177622   
...                        ...               ...               ...   
Wyoming Sweetwater County  ...          1.072643         16.243199   
        Teton County       ...         -1.589565          0.972695   
        Uinta County       ...        -17.755986         -4.916350   
        Washakie County    ...        -11.637475         -0.827815   
        Weston County      ...        -11.752361         -8.040059   

                           RDOMESTICMIG2013  RDOMESTICMIG2014  \
STNAME  CTYNAME                                                 
Alabama Autauga County            -3.012349          2.265971   
        Baldwin County            21.845705         19.243287   
        Barbour County            -7.056824         -3.904

In [25]:
# Now, the key with ony good idiom is to understand when it isn't helping you. In this case, you can actually time both methods and see which on run faster. We can put the approach into a function
# and pass the function into the timeit function to count the time the paramet number allows us to choose how many times we want to run the function. Here we will just set it to 10
def first_approach():
    global df
    return (df.where(df['SUMLEV']==50)
             .dropna()
             .set_index(['STNAME','CTYNAME'])
             .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}))

# Read in our dataset anew
df = pd.read_csv('data/census.csv')

# And now lets run it
timeit.timeit(first_approach, number=10)

0.26023850000001403

In [26]:
# Now let's test the second approach. As you may notice, we use our global variable df in the function. However, changing a global variable inside a function will modify the variable even in a global scope and we
# do not want that to happen in this case. Therefore, for selecting summary levels of 50 only, I create a new dataframe for those records


def second_approach():
    global df
    return df[df['SUMLEV']==50].set_index(['STNAME','CTYNAME']).rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})

# Read in our dataset anew
df = pd.read_csv('data/census.csv')

# And now lets run it
timeit.timeit(second_approach, number=10)

0.07653289999961999

As you can see, the second approach is much faster!. So, this is a particular example of a classic time readability trade off. You'll see lots of examples on stack overflow and in documentatiorn of people using method chaining in their pandas. And so, I think being able to read and understand the syntax is really worth your time, But keep in mind tha following what appears to be stylistic idioms might have performance issues that you need to consider as well.
### Maps
Here's another pandas idiom. Python has wonderful function called map, which is sort of basis for functional programming in the language. When you want to use map in Pyhton, you pass it some funciton you want called, and some iterable, like a list, that you want the funtion to be applied to. The results are that the function is called agains each item in the list, and ther's a resulting list of all of the evaluations of that function. 
Pandas has a similar function called applymap, In applymap, you provide some function which should operate on each cell of DataFrame, and the return set is itself a DataFrame. Now I think applymap is fine, but I actually earely use it. Instead, I find myself often wating to map acroos all of the rows in DataFrame. And pandas has a function that I use heavule ther, called apply. Let' look at example. Let's take a llok at our census DataFrame. In this DataFrame, we have five columns for population estimates with each column corresponding with one year of estimates. It's quite reasonable to want to creat some new columns for minimun or maximun values, and the apply functon is an easy way to do this.
First, we need to write a functon chich takes in a particula row of data, finds a minum and maximun values, and returns a new row of data and returns a new row of data. We'll call this function min_max, this is pretty straight forward. We can create some small slice of a row by projecting the population columns. Then use the Numpy min and max functions, and create a new series with label values represent the new values we want to apply.

In [21]:
def min_max(row):
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    return pd.Series({'min': np.min(data), 'max': np.max(data)})

In [41]:
%%timeit -n 10
# Then we just need to call apply on the DataFrame. Apply tajes the function and the axis on which to operate as parameters. Now, we have to be a bit careful, we've talked about axis zero being the
# rows of the DataFrame in the past. But this parameter is really the parameter of the index to use. So, to apply across all rows, which is applying on all columns, you pass axis equal to columns.
df = pd.read_csv('data/census.csv')
df.apply(min_max, axis = 'columns')

2.67 s ± 289 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
%%timeit -n 10
df = pd.read_csv('data/census.csv')
df[['POPESTIMATE2010',
    'POPESTIMATE2011',
    'POPESTIMATE2012',
    'POPESTIMATE2013',
    'POPESTIMATE2014',
    'POPESTIMATE2015']].apply([np.min, np.max], axis = 'columns')

2.19 s ± 93.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Of course there's no need to limit yourself to returning a new series object. If you're doing this as part of data cleaning your likely to find yourself wanting to add to the existing DataFrame. In that case you just take the row values and add in new columns indicating the max an minimun scores. This is a regular part of my workflow when bringing in data and building summary or descriptive sratistics, and is often used heavuly with the merging of DataFrames.
Here's an exmaple where we have a revised version of the functon min_max Instead of returning a separate series to display the min and max we add two new columns in the original dataframe to store min and max.

In [42]:
def min_max(row):
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    # Create a new entry for max
    row['max'] = np.max(data)
    # Create a new entry for min
    row['min'] = np.min(data)
    return row
# Now just apply the function across the dataframe
df.apply(min_max, axis='columns')

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,max,min
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594,4858979,4785161
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333,55347,54660
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499,203709,183193
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299,27341,26489
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861,22861,22512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,50,4,8,56,37,Wyoming,Sweetwater County,43806,43806,43593,...,-5.339774,-14.252889,-14.248864,1.255221,16.243199,-5.295460,-14.075283,-14.070195,45162,43593
3189,50,4,8,56,39,Wyoming,Teton County,21294,21294,21297,...,19.525929,14.143021,-0.564849,0.654527,2.408578,21.160658,16.308671,1.520747,23125,21297
3190,50,4,8,56,41,Wyoming,Uinta County,21118,21118,21102,...,-6.902954,-14.215862,-12.127022,-18.136812,-5.536861,-7.521840,-14.740608,-12.606351,21102,20822
3191,50,4,8,56,43,Wyoming,Washakie County,8533,8533,8545,...,-2.013502,-17.781491,1.682288,-11.990126,-1.182592,-2.250385,-18.020168,1.441961,8545,8316


In [48]:
# Apply is an extremely importan tool in your toolkit. The reason I introudcen apply is here because you rarely see it used with large function definitions, like we did. Instead, you typically see
# it used with lambdas. To get the most of the discussion you'll see online, you're going to nee to know how to at least read lambdas. 
# Here's you can imagine how you might chain several apply calls with lambdas together toc reat a readable yet succinct data manipulation script. One line example of how youy might calculate the max
# of the columns using the apply function.
rows = ['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013','POPESTIMATE2014', 'POPESTIMATE2015']

df.apply(lambda x: np.max(x[rows]), axis = 1).head()

0    4858979
1      55347
2     203709
3      27341
4      22861
dtype: int64

In [49]:
# The beauty of the apply function is that it allows flexibility in doing whatever manipulating that you desire, as the funciton you pass into apply can be any customized however you want.
# Let's say we want to divide the states into four categories: Northeast, Midwest, South, and West We can write a customized function that returns the region based on the state regions information is
# obtained from Wikipedia
def get_state_region(x):
    northeast = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 
                 'Rhode Island','Vermont','New York','New Jersey','Pennsylvania']
    midwest = ['Illinois','Indiana','Michigan','Ohio','Wisconsin','Iowa',
               'Kansas','Minnesota','Missouri','Nebraska','North Dakota',
               'South Dakota']
    south = ['Delaware','Florida','Georgia','Maryland','North Carolina',
             'South Carolina','Virginia','District of Columbia','West Virginia',
             'Alabama','Kentucky','Mississippi','Tennessee','Arkansas',
             'Louisiana','Oklahoma','Texas']
    west = ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah',
            'Wyoming','Alaska','California','Hawaii','Oregon','Washington']
    
    if x in northeast:
        return "Northeast"
    elif x in midwest:
        return "Midwest"
    elif x in south:
        return "South"
    else:
        return "West"

# Now we have the customized function, Let's say we want to creat a new column called Region, chich shows the state's region, we can use the customized function and the apply function to do so. 
# The customized function is supposed to word on the state name column STNAME. So we will set the apply funtion on the state name column and pass the costumizard function on the state name column
# and pass the customized function into the apply function
df['state_region'] = df['STNAME'].apply(lambda x: get_state_region(x))
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,state_region
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594,South
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333,South
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499,South
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299,South
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861,South


## Group by
Sometimes we want to select data based on groups and understand aggregated data on a group level. We have seen that even though Pandas allows us to iterate over every row in a dataFrame, it is generally very slow to do so. Fortunately Pandas has groupby() function to speed up such task. The idea behind the groupby() function is that it takes some dataframe, splits it into chunks based on some key values, applies computation on those chuncks, then combines the results back together into another dataframe. In pandas this is refered to as the split-apply-combine pattern.
# Splitting

In [53]:
df = pd.read_csv('data/census.csv')
# And exclude state level summarizations, which have sum level value of 40
df = df[df['SUMLEV']==50]
df.head()
# In the first example for groupby() I want to use the census date. Let's get list of the unique states, then we can interate over all the states and for each state we reduce the dataframe and
# calculate the average. Let's run such task for 3 items and time it. For this we'll use cell magic function %%timeit

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


In [64]:
%%timeit -n 3

for state in df['STNAME']. unique():
    avg = np.average(df[df['STNAME'] == state]['CENSUS2010POP'])
    #print("Counties in state {} have an average population of {}".format(state, str(avg)))


31.6 ms ± 3.33 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [68]:
%%timeit -n 3
# You will notice there are two values we set here. groupby() returns a tuple, where the first value is the value of the key were tryung to group by, in this case a specific state name, and the
# second one is projected dataframe that was found for that group
for group, frame  in df.groupby('STNAME'):
    avg = np.average(frame['CENSUS2010POP'])
    #print("Counties in state {} have an average population of {}".format(state, str(avg)))

6.77 ms ± 681 µs per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [74]:
# Wow, what huge difference in speed. An improve by roughly by two factors!. Now, 99% of the time, you'll use group by on one or more columns. But you can also provide a function to group by and use
# that to segment your data. This is a bit of a fabricated example but lets sat that you have a big batch job with lots of processing and you want to work only a third or so of the states at given
# time. We could create some function which returns a number between zero and two based on the first character of the state name. Then we can tell group bu to use this function to split up or data
# frame to be the column that you want to group by first.
# We'll create some new function called set_batch_number and if the first letter of the parameter is a capital M we'll return a 0. If it's a capital Q we'll return a 1 and otherwise we'll return a 2.
# Then we'll pass this funtion to the dataframe.
df.set_index('STNAME', inplace = True)

def set_batch_number(item):
    if item[0] < 'M':
        return 0
    if item[0] < 'Q':
        return 1
    return 2

for group, frame in df.groupby(set_batch_number):
    print('There are ' + str(len(frame)) + ' records in group ' + str(group) + ' for processing.')

# Notice that this time I didn't pass in a column name to groupby(). Instead, I set the index of the dataframe to be STNAME, and if no column indentifier is passed groupby() will automatically use
# the index

There are 1177 records in group 0 for processing.
There are 1134 records in group 1 for processing.
There are 831 records in group 2 for processing.


In [75]:
# Let's take one more look at an example of how we might group data. In this example, I want to use a dataset # of housing from airbnb. In this dataset there are two columns of interest, 
# one is the cancellation_policy and the other is the review_scores_value.
df=pd.read_csv("data/listings.csv")
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [86]:
#df.set_index(["cancellation_policy","review_scores_value"], inplace = True)
for group, frame in df.groupby(level = (0,1)): # We need to pass the leves which we are interested in group by
    print(group)

('flexible', 2.0)
('flexible', 4.0)
('flexible', 5.0)
('flexible', 6.0)
('flexible', 7.0)
('flexible', 8.0)
('flexible', 9.0)
('flexible', 10.0)
('moderate', 2.0)
('moderate', 4.0)
('moderate', 6.0)
('moderate', 7.0)
('moderate', 8.0)
('moderate', 9.0)
('moderate', 10.0)
('strict', 2.0)
('strict', 3.0)
('strict', 4.0)
('strict', 5.0)
('strict', 6.0)
('strict', 7.0)
('strict', 8.0)
('strict', 9.0)
('strict', 10.0)
('super_strict_30', 6.0)
('super_strict_30', 7.0)
('super_strict_30', 8.0)
('super_strict_30', 9.0)
('super_strict_30', 10.0)


In [87]:
# This seems to work ok. But what if we wanted to group by the cancelation policy and review scores, but separate out all the 10's from those under ten? In this case, we could use a function 
# to manage the groupings

def grouping_fun(item):
    # Check the "review_scores_value" portion of the index. item is in the format of cancellation_policy,review_scores_value
    if item[1] == 10.0:
        return (item[0],"10.0")
    else:
        return (item[0],"not 10.0")

for group, frame in df.groupby(by=grouping_fun):
    print(group)

('flexible', '10.0')
('flexible', 'not 10.0')
('moderate', '10.0')
('moderate', 'not 10.0')
('strict', '10.0')
('strict', 'not 10.0')
('super_strict_30', '10.0')
('super_strict_30', 'not 10.0')


## Applying
To this point we have applied very simple processing to our data after splitting, really just outputting some print statements to demostrate how the splitting works. The pandas developers have three broad categories of data processing to happen during the apply step: Aggregation of group data, transformation of gruoup data, and filtration of group data
### Aggregation
The most straingh forward apply step is the aggregation of data, and uses the method agg() on the groupby object. Thus far we have onlu iterated through the groupby object, unpacking in into a label (the group name) and a dataframe. but with agg we can pass in a dictionary of the columns we are interested in aggregating along with the function we are looking to apply to aggregate.

In [95]:
#df.reset_index(inplace = True)
#Now let's group by the cancellation policy and find the average review_scores_value by group
df.groupby('cancellation_policy').agg({'review_scores_value': np.nanmean}) # Uses nanmean for work with nan values
# We can just extend this dictionary to aggregate by multiple functions or multiple columns.
df.groupby("cancellation_policy").agg({"review_scores_value":(np.nanmean,np.nanstd), "reviews_per_month":np.nanmean})
# Take a moment to make sure you understand the previous cell, since it's somewaht complex. First we're doing a group by on the dataframe object by the column "cancellation_policy". This creates a 
# new GroupBy object. Then we are invoking the agg() function on that object. The agg function is going to apply one or more functions we specify to the group dataframes and return a single row per
# dataframe/group. When we called this function we sent it two dictionary entries, each with the key indicating which column we wanted functions applied to. For this first column we actually supplied
# a tuple of two functions. Note that these are not function invocations, like np.nanmean(), or function names, like "nanmean" they are references to functions which will return single values.
# The groupby object will recognize the tuple and call each function in order on the same column. The results willbe in a heirarchical index, bu since they are columns they don't show as index per se
# Then we indcated another column and a single function we wanted to run.

review_scores_value           reviews_per_month
                                nanmean    nanstd           nanmean
cancellation_policy                                                
flexible                       9.237421  1.096271          1.829210
moderate                       9.307398  0.859859          2.391922
strict                         9.081441  1.040531          1.873467
super_strict_30                8.537313  0.840785          0.340143

### transformation 
Transformation is different from aggreation. Where agg() returns a single value per column, so one row per group, transform() returns an object that is the same size as the group. Essentially, it broadcasts the function you supply over the grouped dataframe, returning a new dataframe This makes combinign data later easy.
For instance, suppose we want to include the averga rating values in a given group by cancellation policity, but preserve the dataframe shape so that we could generate a difference between an individual obseration an the sum.

In [98]:
# First, lets define just some subset of columns we are interested in
cols = ['cancellation_policy', 'review_scores_value']
# now let's transform it, I'll store this in its own dataframe
transform_df = df[cols].groupby('cancellation_policy').transform(np.nanmean)
transform_df.head() # Return a new dataframe with same size as original group, it make easy other operations. agg only return one element for column

,review_scores_value
0,9.307398
1,9.307398
2,9.307398
3,9.307398
4,9.237421


In [102]:
# So we can see that the index here is actually the same as the original dataframe. So lets just join this in. Before we do that, lets rename the column in the transformed version
transform_df.rename({'review_scores_value':'mean_review_scores'}, axis='columns', inplace=True)
df=df.merge(transform_df, left_index=True, right_index=True)
df.head()

,level_0,index,cancellation_policy,review_scores_value,id,listing_url,scrape_id,last_scraped,name,summary,...,review_scores_location,requires_license,license,jurisdiction_names,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores
0,0,0,moderate,NaN,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",...,NaN,f,NaN,NaN,f,f,f,1,NaN,9.307398
1,1,1,moderate,9.0,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,...,9.0,f,NaN,NaN,t,f,f,1,1.30,9.307398
2,2,2,moderate,10.0,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...",...,9.0,f,NaN,NaN,f,t,f,1,0.47,9.307398
3,3,3,moderate,10.0,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,...,10.0,f,NaN,NaN,f,f,f,1,1.00,9.307398
4,4,4,flexible,10.0,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...",...,9.0,f,NaN,NaN,f,f,f,1,2.25,9.237421


### Filtering
The group by object has build in support for filtering groups as well. It's often that you'll want to group by some feature, then some transformation to the groups, the drop certain groups as part of your cleaning reoutines. the filter() function takes in a function chich it applies to each group dataframe and returns either a True or a Flase, depending upon whether that group should be inclued in the results. For instance, if we only want those groups which have a mean rating above 9 inclued in our results

In [107]:
# For instance, if we only awant those groups which have a mean rating above 9 included in our results
df.groupby('cancellation_policy').filter(lambda x: np.mean(x['review_scores_value'])>9.2).head()
# Notice that the results are still indexed, but that any of the results which were in a group with a mean review score of less than or equal to 9.2 were not copied over

,level_0,index,cancellation_policy,review_scores_value,id,listing_url,scrape_id,last_scraped,name,summary,...,review_scores_location,requires_license,license,jurisdiction_names,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores
0,0,0,moderate,NaN,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",...,NaN,f,NaN,NaN,f,f,f,1,NaN,9.307398
1,1,1,moderate,9.0,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,...,9.0,f,NaN,NaN,t,f,f,1,1.30,9.307398
2,2,2,moderate,10.0,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...",...,9.0,f,NaN,NaN,f,t,f,1,0.47,9.307398
3,3,3,moderate,10.0,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,...,10.0,f,NaN,NaN,f,f,f,1,1.00,9.307398
4,4,4,flexible,10.0,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...",...,9.0,f,NaN,NaN,f,f,f,1,2.25,9.237421


### Applying
By far the most common operation I invoke on groupby objects is the apply function. This allows you to apply an arbitrary function to each group, and stitch the results back for each apply() into a single dataframe where the index is preserved. Let's look at an example using our airbnb data, I'm going to get a clean vopy of the dataframe

In [108]:
df=pd.read_csv("data/listings.csv")
# And lets just include some of the columns we were interested in previously
df=df[['cancellation_policy','review_scores_value']]
df.head()

,cancellation_policy,review_scores_value
0,moderate,NaN
1,moderate,9.0
2,moderate,10.0
3,moderate,10.0
4,flexible,10.0


In [111]:
# In previous work we wanted to find the average review score of a listing and its deviation from the group mean. This was a two process, first we use transform() on the groupby object and then we 
# had to broadcast to creat a new column. With apply() we could wrap this logic in one place
def calc_mean_review_scores(group):
    avg = np.nanmean(group['review_scores_value'])
    group["review_scores_mean"]=np.abs(avg-group["review_scores_value"])
    return group
df.groupby('cancellation_policy').apply(calc_mean_review_scores).head()

,cancellation_policy,review_scores_value,review_scores_mean
0,moderate,NaN,NaN
1,moderate,9.0,0.307398
2,moderate,10.0,0.692602
3,moderate,10.0,0.692602
4,flexible,10.0,0.762579


In [113]:
# Using apply can be slower than using some of the specialized functions, especially agg(). But, if your dataframes are not huge, it's a solid general purpose approach

# Scales

In [115]:
# Here's an example. Lets create a dataframe of letter grades in descending order. We can also set an index value and here we'll just make it some human judgement of how good a student was, like
# "excellent" or "good"

df = pd.DataFrame(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D'], index = ['excellent', 'excellent', 'excellent', 'good', 'good', 'good', 'ok', 'ok', 'ok', 'poor', 'poor'],
                  columns = ['Grades'])
df

,Grades
excellent,A+
excellent,A
excellent,A-
good,B+
good,B
good,B-
ok,C+
ok,C
ok,C-
poor,D+


In [118]:
# Now, if we checj the datatype of this column, we see that it's just an object, since we set string values
print(df.dtypes)
# We can, however, tell pandas that we want to change the type to category, using the astype() function
print(df['Grades'].astype("category").head())
# We see now that there are eleven categories, and pandas is aware of what those categories are. More interesting though is that our data isn't just categorical, but that it's ordered. That is, an A-
# come after a B+, and B comes before a B+. We can tell pandas that the data is ordered by first creating a new categorical data type with the list of the categories (in order) and the ordered = 
# true flag
my_categories = pd.CategoricalDtype(categories = ['D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+'], ordered=True)
grades = df['Grades'].astype(my_categories)
grades.head()

Grades    object
dtype: object
excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): ['A', 'A+', 'A-', 'B', ..., 'C+', 'C-', 'D', 'D+']


excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']

Now we see that pandas is not only aware that there are 11 categories, but it is also aware of the order of those categoreies. So, what can you do with this? Well because there is an ordering this can help with comparisons and boolean masking. For instance, if we have a list of our grades and we compare them to a "C" we see that the lexicographical comparison returns results we were not intending

In [121]:
print(df[df['Grades']>'C'])
# So a C+ is grat than a C, but a C- and D certinly are not. However, if we broadcast over the dataframe which has the type set to an ordered categorical 
print(grades[grades>'C'])

     Grades
ok       C+
ok       C-
poor     D+
poor      D
excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']


We see that the operator works as we would expect. We can the use a certain set of mathematical operators, like minimun, maximun, etc., on the ordinal data. Sometime it is useful to represent categorical values as each being a column with a true or a false as to whether the category applies. This is especially common in feature extraction, which is a topic in the data mining course. Variables with a boolean value are typically called dummy variables, and pandas has a built in function called get_dummies chich will convert the values of a single column into multiple columns of zeros and ones indicating the presence of the dummy variable. I rarely use it, but when I do it's very handy.
There's one more common scale-based operation I'd like to talk about, and that's on converting a scale from something that is on the interval or ratio scale, like a numeric grade, into one chich is categorical. Now, this might seem a bit counter intuitive to you, since you are losing information about the value. But it's this can be an extremely useful approach, and histograms are regularly used with converted inverval or ratio data. In addition, if you're usign a machine learning classification approach on data, you need to be using categorical data, so reducing dimensionality may be useful just to apply a given technique. Pandas has a fucntion called cut which takes as an argument some array-like structure like a column of a dataframe or a series. It also takes a number of bins to used and all bins are kept at equal spacing. Lets goback to our census data for an example. We saw that we could group by state, the aggregate to get a list of the average county size by state. If we futher apply cut to this with, say, ten bins, we can see the states listed as categoricals using the average county size.

In [122]:
# Now we read in our dataset
df=pd.read_csv("data/census.csv")

# And we reduce this to country data
df=df[df['SUMLEV']==50]

# And for a few groups
df=df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg(np.average)

df.head()

STNAME
Alabama        71339.343284
Alaska         24490.724138
Arizona       426134.466667
Arkansas       38878.906667
California    642309.586207
Name: CENSUS2010POP, dtype: float64

In [125]:
pd.cut(df, 10) # Cut values in 10 groups

STNAME
Alabama                   (11706.087, 75333.413]
Alaska                    (11706.087, 75333.413]
Arizona                 (390320.176, 453317.529]
Arkansas                  (11706.087, 75333.413]
California              (579312.234, 642309.586]
Colorado                 (75333.413, 138330.766]
Connecticut             (390320.176, 453317.529]
Delaware                (264325.471, 327322.823]
District of Columbia    (579312.234, 642309.586]
Florida                 (264325.471, 327322.823]
Georgia                   (11706.087, 75333.413]
Hawaii                  (264325.471, 327322.823]
Idaho                     (11706.087, 75333.413]
Illinois                 (75333.413, 138330.766]
Indiana                   (11706.087, 75333.413]
Iowa                      (11706.087, 75333.413]
Kansas                    (11706.087, 75333.413]
Kentucky                  (11706.087, 75333.413]
Louisiana                 (11706.087, 75333.413]
Maine                    (75333.413, 138330.766]
Maryland     

In [126]:
# Here we see that states like alabama and alaska fall into the same category, while california and the disctrict of columnbia fall in a very different category. Now, cutting is just one way to
# build categories from your data, and there are many other methods. For instance, cut gives you interval data, where the spacing between each category is equal sized. but sometimes you want
# to from categories bases on frecquency - you want the number of items in each bin to the be the same, instead of the spacing between bins. It really depens on what the shape of your data is, and 
# what you're planning to do with it.

# Pivot Table
A pivot table is a way of summatizing data in a DataFrame for a particular purpose. It makes heavy use of the aggregation function. A pivot table is itself a Dataframe, where the rows represent one variable that you're interested in, the columns another, and the cell's some aggregate value. A pivot table also tends to incluedes marginal values as well, which are the sums for each column and row. This allows you to be able to see the relationship between two variables at just a glance.

In [127]:
# Here we have the Times Higher Education World University Ranking dataset, which is one of the most influential university measures. Let's import the dataset and see what it looks like
df = pd.read_csv('data/cwurData.csv')
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [128]:
# Here we can see each institutiopn's rank, country, quality of education, other metrics, and overall score. Let's say we want to create a new column called Rank_Level, where institutions with world
# ranking 1-100 are categorized as first tier and those with world ranking 101 - 200 are second tier, ranking 201 -300 are third tier, after 301 is other top universities.
def create_category(ranking):
    # Since the rank is just an integer, I'll just do a bunch of if/elif statements
    if (ranking >= 1) & (ranking <= 100):
        return "First Tier Top Unversity"
    elif (ranking >= 101) & (ranking <= 200):
        return "Second Tier Top Unversity"
    elif (ranking >= 201) & (ranking <= 300):
        return "Third Tier Top Unversity"
    return "Other Top Unversity"
df['Rank_Level'] = df['world_rank'].apply(lambda x: create_category(x))
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,Rank_Level
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,First Tier Top Unversity
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,First Tier Top Unversity
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,First Tier Top Unversity
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,First Tier Top Unversity
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,First Tier Top Unversity


In [132]:
# A pivot table allows us to pivot out one of these columns a new column headers and compare it against another column as row indices. Let's say we wanto to caompare rank level versus country of the
# universities and we want to compare in terms of overll score. To do this, we tell Pandas we want the values to be Score, and index to be the country and the columns to be the rank levels. Then
# we specify that the aggregation function, and here we'll use the Numpy mean to get the average rating for universities in tha country
df.pivot_table(values = 'score', index = 'country', columns = 'Rank_Level', aggfunc = [np.nanmean]).head()
# We can see a hierarchical dataframe where the index, or rows, are by country and the columns have two levels, the top level indicating that the mean value is being used and the second level being
# our ranks. In this example we only have one variable, the mean, that we are looking at, so we don't really need a hierarchical index. We notice that there are somne nan values, for example the
# first row, Argentina. The Nan values indicate that argentina has only observations in "The other top universities" category

nanmean                      \
Rank_Level First Tier Top Unversity Other Top Unversity   
country                                                   
Argentina                       NaN           44.672857   
Australia                   47.9425           44.645750   
Austria                         NaN           44.864286   
Belgium                     51.8750           45.081000   
Brazil                          NaN           44.499706   

                                                               
Rank_Level Second Tier Top Unversity Third Tier Top Unversity  
country                                                        
Argentina                        NaN                      NaN  
Australia                    49.2425                47.285000  
Austria                          NaN                47.066667  
Belgium                      49.0840                46.746667  
Brazil                       49.5650                      NaN

In [133]:
# Now pivot tables aren't limited to one function that you might want to apply. You can pass a named parameter, aggfunc, which is a list of the different functions to apply, and pandas will 
# provide you with the result using hierarchical column names. Let's try that same query, but áss ion the max function too
df.pivot_table(values = 'score', index = 'country', columns = 'Rank_Level', aggfunc = [np.nanmean, np.max]).head()

nanmean                      \
Rank_Level First Tier Top Unversity Other Top Unversity   
country                                                   
Argentina                       NaN           44.672857   
Australia                   47.9425           44.645750   
Austria                         NaN           44.864286   
Belgium                     51.8750           45.081000   
Brazil                          NaN           44.499706   

                                                               \
Rank_Level Second Tier Top Unversity Third Tier Top Unversity   
country                                                         
Argentina                        NaN                      NaN   
Australia                    49.2425                47.285000   
Austria                          NaN                47.066667   
Belgium                      49.0840                46.746667   
Brazil                       49.5650                      NaN   

                               amax                      \
Rank_Level First Tier Top Unversity Other Top Unversity   
country                                                   
Argentina                       NaN               45.66   
Australia                     51.61               45.97   
Austria                         NaN               46.29   
Belgium                       52.03               46.21   
Brazil                          NaN               46.08   

                                                               
Rank_Level Second Tier Top Unversity Third Tier Top Unversity  
country                                                        
Argentina                        NaN                      NaN  
Australia                      50.40                    47.47  
Austria                          NaN                    47.78  
Belgium                        49.73                    47.14  
Brazil                         49.82                      NaN

In [134]:
# So now we see we have both the man an the max. As mentioned earlier, we can also summarize the values within a given top level column. For instance, if we want to see an overall averange for the
# mean and we ant see the max of the max, we can indicate that we want pandas to provide marginal values
df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max], margins=True).head()

mean                      \
Rank_Level First Tier Top Unversity Other Top Unversity   
country                                                   
Argentina                       NaN           44.672857   
Australia                   47.9425           44.645750   
Austria                         NaN           44.864286   
Belgium                     51.8750           45.081000   
Brazil                          NaN           44.499706   

                                                                          \
Rank_Level Second Tier Top Unversity Third Tier Top Unversity        All   
country                                                                    
Argentina                        NaN                      NaN  44.672857   
Australia                    49.2425                47.285000  45.825517   
Austria                          NaN                47.066667  45.139583   
Belgium                      49.0840                46.746667  47.011000   
Brazil                       49.5650                      NaN  44.781111   

                               amax                      \
Rank_Level First Tier Top Unversity Other Top Unversity   
country                                                   
Argentina                       NaN               45.66   
Australia                     51.61               45.97   
Austria                         NaN               46.29   
Belgium                       52.03               46.21   
Brazil                          NaN               46.08   

                                                                      
Rank_Level Second Tier Top Unversity Third Tier Top Unversity    All  
country                                                               
Argentina                        NaN                      NaN  45.66  
Australia                      50.40                    47.47  51.61  
Austria                          NaN                    47.78  47.78  
Belgium                        49.73                    47.14  52.03  
Brazil                         49.82                      NaN  49.82

In [136]:
# A pivot table is just a multi-level dataframe and we can access series or cells in the datafrmae in a similar way as we do so for a regular dataframe
new_df=df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max], margins=True)
# Now let's look at the index
print(new_df.index)
# And let's look at the columns
print(new_df.columns)
# We can see the columns are hierarchical. The top level column indices have two categories: mean and max, and the lower level column indices have four categories, chich are the four rank leves.
# How would we query this if we want to get the average scores of first tier top university levels in each country? We would just need to maje two dataframe projections, the first for the mean,
# then the second for the top tier
new_df['mean']['First Tier Top Unversity'].head()

Index(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria',
       'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Iran',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia',
       'Serbia', 'Singapore', 'Slovak Republic', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand',
       'Turkey', 'USA', 'Uganda', 'United Arab Emirates', 'United Kingdom',
       'Uruguay', 'All'],
      dtype='object', name='country')
MultiIndex([('mean',  'First Tier Top Unversity'),
            ('mean',       'Other Top Unversity'),
            ('mean', 'Second Tier Top Unversity'),
            ('mean',  'Third T

country
Argentina        NaN
Australia    47.9425
Austria          NaN
Belgium      51.8750
Brazil           NaN
Name: First Tier Top Unversity, dtype: float64

In [142]:
# What if we want to find the country that has the maximun average score on First Tier Top University level?
# We can use the idexmax() function
new_df['mean']['First Tier Top Unversity'].idxmax()
# Now, the idxmax() function isn't special for pivot tables. If you want to achieve a different shape of your pivot table, you can do so with the stack and unstack function. Stacking is pivoting the
# lowermost column index to vecome the innermost row index. Unstacjking is the inverse of stacking, pivotinf the innermost row index to become the lowermost column index. An example will help make
# this clear

'United Kingdom'

In [151]:
new_df.stack().head()# The columns tier convert in lower index in rows

mean   amax
country   Rank_Level                                 
Argentina Other Top Unversity        44.672857  45.66
          All                        44.672857  45.66
Australia First Tier Top Unversity   47.942500  51.61
          Other Top Unversity        44.645750  45.97
          Second Tier Top Unversity  49.242500  50.40

In [152]:
new_df.unstack().head()# The columns tier convert in lower index in rows

      Rank_Level                country  
mean  First Tier Top Unversity  Argentina        NaN
                                Australia    47.9425
                                Austria          NaN
                                Belgium      51.8750
                                Brazil           NaN
dtype: float64

In [155]:
new_df.unstack() # the rows convert in columns in one column with multiple indexes and only one value

      Rank_Level                country             
mean  First Tier Top Unversity  Argentina                    NaN
                                Australia                47.9425
                                Austria                      NaN
                                Belgium                  51.8750
                                Brazil                       NaN
                                                          ...   
amax  All                       Uganda                   44.4000
                                United Arab Emirates     44.3600
                                United Kingdom           97.6400
                                Uruguay                  44.3500
                                All                     100.0000
Length: 600, dtype: float64

# Timestamp
In today's lecture, where we'll be looking at the time series and date functionally in pandas. Manipulating dates and time is quite flexible in Pandas and thus allows us to conduct more analysis such as time series analysis, which we will talk about soon. Actually, pandas was originally created by Wed MecKinney to handle data and time data when worked as a consultant for hedge funds.
Pandas has four main time related classes. Timestamp, DatatimeIndex, Period, and PeriodIndex. First, let's look a timestamp. It represents a single timestamp using a string 9/1/2019 10:05AM, and here we have iour timestamp. Timestamp is interchangable with python's datetime in most cases.

In [6]:
print(pd.Timestamp('9/1/2019 10:05AM'))
print(pd.Timestamp('10:05')) # With today
print(pd.Timestamp(2019, 12, 20, 0, 0)) # Pass year,month, day, etc
print(pd.Timestamp(2019, 12, 20, 0, 0).isoweekday()) # Timestamp also has some useful attributes, cuch as isoweekday(), which shows the weekday of the timestamp, note tha 1 reprsents Monday and 7 Sunday
print(pd.Timestamp(2019, 12, 20, 5, 2,23).second) # You can find extract the specific year, month, day, hour, minute, second from a timestamp

2019-09-01 10:05:00
2022-05-02 10:05:00
2019-12-20 00:00:00
5
23


## Period
Suppose we weren't interested in a specific point in time and instead wanted a span of time. This is where the period class comes into play. Period represents a single time span, such as a specific day or month. He we are creating a preiod that is January 2016

In [11]:
print(pd.Period('1/2016'))
# You will notice when we print that out the granualarity of the period is M for month, since that was the finest graines pice we provided. Here's an example of period that is March 5th, 2016.
print(pd.Period('3/5/2016'))
# Period object represent the full timespan that you specify. Arithmetic on period is very easy and intuitive, for instance, if we want to find out 5 months after January 2016, we simply plus 5
print(pd.Period('1/2016') + 5)
# From the result, you can see we get June 2016. If we want to find out two days before March 5th 2016, we simply subtract 2
print(pd.Period('3/5/2016') - 2)
# The key here is that the period object encpsulates the granualarity for arithmetic

2016-01
2016-03-05
2016-06
2016-03-03


## DatetimeIndex and PeriodIndex
The index of timestamp is DatetimeIndex. Let's look at quich example. First, let0s create our example series t1, we'll use the timestamp of september 1st, 2nd and 3rd of 2016. When we look at the series each timestamp is the index and has a value associated with it. in this case, a, b, c.

In [15]:
t1 = pd.Series(list('abc'), [pd.Timestamp('2016-09-01'), pd.Timestamp('2016-09-02'), pd.Timestamp('2016-09-03')])
print(t1)
print(type(t1.index))
# Similarly, we can create a period-based index as well. 
t2 = pd.Series(list('def'), [pd.Period('2016-09'), pd.Period('2016-10'), pd.Period('2016-11')])
print(t2)
print(type(t2.index))

2016-09-01    a
2016-09-02    b
2016-09-03    c
dtype: object
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
2016-09    d
2016-10    e
2016-11    f
Freq: M, dtype: object
<class 'pandas.core.indexes.period.PeriodIndex'>


## Converting to Datetime
Now let's look into how to convert to Datetime. Suppose we have a list of dates as strings and we want to create a new dataframe

In [17]:
d1 = ['2 June 2013', 'Aug 29, 2014', '2015-06-26', '7/12/16']
ts3 = pd.DataFrame(np.random.randint(10, 100, (4,2)), index=d1, columns=list('ab'))
# Using pandas to_datetime, pandas will try to convert these to Datetime and put them in a standard format.
ts3.index = pd.to_datetime(ts3.index)
ts3

,a,b
2013-06-02,66,17
2014-08-29,35,27
2015-06-26,40,57
2016-07-12,82,40


In [18]:
# to_datetime also() has options to change the date parse order. For example, we can pass in the argument dayfirst = True to parse the date in European date.
pd.to_datetime('4.7.12', dayfirst=True)

Timestamp('2012-07-04 00:00:00')

## Timedelta and Offset
Timedeltas are differences in times. This is not same as a period, but conceptually similar. For instacen, if we want to taje the difference between september 3rd and september 1st, we get a timedelta of two days.
Offset is similar to timedelta, but it followes specific calendar duration rules. Offset allows flecibility in terms of types of time intervals. Besides hour, bay, week, month, etc. It also has business day, end of month, semi month begin etc.

In [23]:
print(pd.Timestamp('9/3/2016')-pd.Timestamp('9/1/2016'))
# We can also do something like find what the date and time is for 12 days and three hours past September 2nd, at 8:10 AM.
print(pd.Timestamp('9/2/2016 8:10AM') + pd.Timedelta('12D 3H'))

2 days 00:00:00
2016-09-14 11:10:00


In [30]:
print(pd.Timestamp('9/4/2016').weekday())
# Now we can now add the timestamp with a week ahead
print(pd.Timestamp('9/4/2016') + pd.offsets.Week())
# Now let's try to do the month end, then we would have the last day of Septemer
print(pd.Timestamp('9/4/2016') + pd.offsets.MonthEnd())

6
2016-09-11 00:00:00
2016-09-30 00:00:00


## Working with Dates in a Dataframe
Next, let's look at a few tricks for working with dates in a DataFrame. Suppose we want to llok at nine measurements, taken bi weekly, every Sunday, starting in October 2016. Using date_rage, we can create this DatetimeIndex. In data_range, we have to either specify the start or end date. If it is not explicitly specified, by default, the date is considered the start date. Then we have to specify number of periods, and a frequency. Here, we set it to '2W-SUN', which means biweekly on Sunday.

In [35]:
dates = pd.date_range('10-01-2016', periods = 9, freq = '2W-SUN')
print(dates)
# There are many other frequencies that you can specify. For example, you can do business day
print(pd.date_range('10-01-2016', periods=9, freq='B'))
# Or you can do quarterly, with the quarter start in June
print(pd.date_range('04-01-2016', periods=12, freq='QS-JUN'))
# Now, Let's go back to our weekly on Sunday example and create a DataFrame using these dates, and some random data, and see what we can do with it
dates = pd.date_range('10-01-2016', periods=9, freq = '2W-SUN')
df = pd.DataFrame({'Count 1': 100 + np.random.randint(-5, 10, 9).cumsum(), 'Count 2': 120 + np.random.randint(-5, 10, 9)}, index = dates)
df

DatetimeIndex(['2016-10-02', '2016-10-16', '2016-10-30', '2016-11-13',
               '2016-11-27', '2016-12-11', '2016-12-25', '2017-01-08',
               '2017-01-22'],
              dtype='datetime64[ns]', freq='2W-SUN')
DatetimeIndex(['2016-10-03', '2016-10-04', '2016-10-05', '2016-10-06',
               '2016-10-07', '2016-10-10', '2016-10-11', '2016-10-12',
               '2016-10-13'],
              dtype='datetime64[ns]', freq='B')
DatetimeIndex(['2016-06-01', '2016-09-01', '2016-12-01', '2017-03-01',
               '2017-06-01', '2017-09-01', '2017-12-01', '2018-03-01',
               '2018-06-01', '2018-09-01', '2018-12-01', '2019-03-01'],
              dtype='datetime64[ns]', freq='QS-JUN')


,Count 1,Count 2
2016-10-02,104,115
2016-10-16,104,125
2016-10-30,99,126
2016-11-13,101,128
2016-11-27,97,117
2016-12-11,101,121
2016-12-25,97,117
2017-01-08,106,120
2017-01-22,108,118


In [43]:
# First, we can check what day of the week a specific date is. For example, here we can see that all the dates in our index are on a Sunday. Which matches the freequency that we set
print(df.index.weekday)
# We can also use diff() to find the difference between each date's value.
df.diff()

Int64Index([6, 6, 6, 6, 6, 6, 6, 6, 6], dtype='int64')


,Count 1,Count 2
2016-10-02,NaN,NaN
2016-10-16,0.0,10.0
2016-10-30,-5.0,1.0
2016-11-13,2.0,2.0
2016-11-27,-4.0,-11.0
2016-12-11,4.0,4.0
2016-12-25,-4.0,-4.0
2017-01-08,9.0,3.0
2017-01-22,2.0,-2.0


In [44]:
# Suppose we want to know what the mean count is for each month in our DataFRame. We can do this using resample. Converting from a higher frequency from a lower frequency is called downsapling (we'll 
# talk about this in a moment)
df.resample('M').mean()

,Count 1,Count 2
2016-10-31,102.333333,122.0
2016-11-30,99.000000,122.5
2016-12-31,99.000000,119.0
2017-01-31,107.000000,119.0


In [46]:
# Now let's talk about datatime indexing and slicing, which is a wonderfull feature of the pandas DataFrame. For instance, we can use partial strinf indxing to find values from a particular year.
df.loc['2017']

,Count 1,Count 2
2017-01-08,106,120
2017-01-22,108,118


In [47]:
# Or we can do it from a particular month
df.loc['2016-12']

,Count 1,Count 2
2016-12-11,101,121
2016-12-25,97,117


In [48]:
# Or we can even slice on a range of dates For example, here we only want the values from December 2016 onwards.
df['2016-12':]

,Count 1,Count 2
2016-12-11,101,121
2016-12-25,97,117
2017-01-08,106,120
2017-01-22,108,118
